In [129]:
import pandas as pd
import numpy as np
import csv 
import matplotlib.pyplot as plt
import spacy
import math
import os
from src.utils import *

os.chdir('/Users/ismailakrout/Desktop/python/statapp')

pd.set_option("display.max_columns", None)


In [130]:
dtypes = {
    'party.facts.id' : str,
    'date': object,
    'agenda': object,
    'speechnumber': int,
    'speaker': object,
    'party': object,
    'party.facts.id': object,
    'chair': bool,
    'terms': int,
    'text': object,
}

In [131]:
df = read_input('00. input/Corp_HouseOfCommons_V2_2010-2.csv', encod='ISO-8859-1', dtype_values=dtypes)
df.drop(columns=['Unnamed: 0', 'iso3country', 'parliament', 'party.facts.id', 'speechnumber', 'chair', 'terms', 'date'], inplace=True)
df.rename(columns=
    {'speaker': 'Speaker'},
    inplace=True
)

In [132]:
# pour faire des essais rapides on réduit la taille du DataFrame

df = df.sample(frac=1).reset_index(drop=True)
df = df.head(1000)


In [133]:
# on ne garde que Labor et Conservative
df = df.loc[df['party'].isin(['Lab', 'Con'])]

## Data processing

On va tout d'abord créer une fonction qui va appliquer le même traitement aux speeches qui a été appliqué aux textes des journaux : 
- tokenization 
- nettoyage de la ponctuation et des nombres 
- stemming des mots 
- nettoyage des stopwords à l'aide de la même liste que pour les journaux
- remise des mêmes stopwords significatifs que pour les journaux 

In [134]:
from spacy.lang.en import English
nlp = spacy.load('en_core_web_sm')
nlp.disable_pipes(["tagger", "parser"])
import string
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')

On utilise le SnowballStemmer.

On importe la liste des stopwords :

In [135]:
df_stopwords = pd.read_csv("00. input/sw1k.csv",
                names=['word', 'frequency', 'presence', 'doc_size_sum', 'type'],
                encoding='latin-1').drop(index=0)

On a donc un dataframe des stopwords. On en tire une liste simple des stopwords, où l'on remet cependant des stopwords jugés significatifs, comme expliqué pour le traitement des journaux :

In [136]:
stopwords = df_stopwords['word'].unique()
putback = ['prime', 'officials', 'security', 'news', 'working', 'games', 'jobs', 'campaign', 'services',
'civil', 'economic', 'information', 'political', 'election', 'court', 'office', 'vote', 'trump', 'control', 'job', 'price',
'donald trump', 'chinese', 'problems', 'concerns', 'minister', 'nation', 'policy', 'data', 'indian', 'congress',
'president', 'network', 'american', 'accused', 'government', 'money', 'investigation',
'facebook', 'success', 'prices', 'twitter', 'book', 'politics',
'justice', 'claims', 'russia', 'law', 'technology', 'content', 'union', 'european', 'workers']

stopwords = list(set(stopwords) - set(putback))


On utilise la librairie SpaCy comme pour le traitement des journaux, et on enlève les nombres et la ponctuation avec la méthode translate. 

In [137]:
def clean(text):
  text = str(text).lower()
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = text.translate(str.maketrans('', '', string.digits)) 
  doc = nlp(text)
  l = []
  for word in doc :
    word = str(word)
    l.append(stemmer.stem(word))
    if l[-1] == ' ' or l[-1] in stopwords: 
      l.pop()
  return l

In [ ]:
df['agenda'] = df['agenda'].apply(clean)

In [147]:
df['agenda']

0                      {union, european, referendum}
1                                           {financ}
2      {crime, oral, solicitorgener, retail, answer}
3                              {communiti, pharmaci}
4                                           {mental}
                           ...                      
994                                          {polic}
995                             {nhs, govern, spend}
996                               {joint, enterpris}
997                                           {gaza}
998                             {regul, educ, faith}
Name: agenda, Length: 781, dtype: object

In [139]:
# df['agenda'] = df['agenda'].apply(set)

On peut ensuite appliquer ce nettoyage à nos speeches, et aux sujets de ceux-ci, directement dans le dataframe. 

Maintenant, chaque speech est transformé en une liste de mots significatifs. 

On voit par exemple que 24 speeches de l'échantillon ont "data" répertorié comme sujet. 

# **Construction du test**

On va tout d'abord essayer de filtrer les speeches qui sont dans le "thème" des nouvelles technologies, big tech, etc. Pour ça, on se créé une liste de mots clés à partir des mots clés utilisés pour les journaux, et on ne garde que les speeches qui sont concernés par ces sujets. On commence par faire une liste assez grossière. 

In [140]:
technology=['technology','innovat','computer','high tech|high-tech','science','engineering']
consumer_protection=['privacy','data leak','leak','fake news',' safety','decept','defective','hack']
firms=['google','alphabet','apple','facebook','meta','amazon','microsoft']
products=['chrome','incognito','youtube','nexus','pixel','google drive','gmail','glass','street view','buzz','fitbit',
 'maps', 'doodle','play','translate','search', 'google news','nest hub','xl','nest','chromecast','stadia','hub',
 'marshmallow','lollipop','cloud','waymo','earth','engine',

'apple pay','apple watch','iphone','ipad','ipod','iwatch','macbook','macbook pro', 'macbook air','mac',
 'imac','airpods','ios','siri','icloud','apple tv','apple music','app store', 'safari','x','app','apple store',
 'xr', 'xs', 'se','iphones','itunes','ibook','plus','pro','max','mini','os','airtag','airtags','arcade','homepod',
'keynote','ipados','id','foxconn','facetime','beat','stalk',

'messenger','instagram', 'whatsapp','page','feed','oculus',

'prime', 'kindle',
'publishing','amazon prime','amazon drive','amazon video','amazon business','amazon web service',
'amazon cloud', 'alexa','echo dot','echo','dot', 'delivery', 'amazon uk','unlimited', 'episode','foods','grocery', 
'grand tour','grand','tour','viking', 'vikings','argo','argos','macmillan', 'dvd','clarkson','lord','ring','hair','skin','vacuum',
'pre','beer','drake','spark','kart','dog','twitch','cat','xo','matthew','stafford','ratchet','clank',
'swagway','album','mouse','showbiz','beauty','guardian','batman','arkham','gc','hair','skin','shirt',
'lovefilm','mirzapur','cast','audio','drama','movie','jack ryan','actor','character','lucifer','outlander',
'premier','super mario','sky','channel','voyage',


'windows','window','xp','surface','xbox','studio','microsoft office', 'office','word','cortana', 'surface pro','teams',  'playstation',
'microsoft edge', 'edge', 'gear','outlook','halo','skype','kinect','internet explorer','explorer','ie','bing','xcloud','hololens',
'forza','ori','scarlett','scorpio','wordperfect','valhalla','onedrive','games gold','lumia','azure',
'assassin creed','assassin','creed','minecraft','yammer','warcraft','tay']



ceos=['sundar','pichai','eric','schmidt','steve jobs','tim cook','mark zuckerberg','andy jassy','jeff','bezos','satya', 'nadella','bill gates',
      'gates','steve job','steve','tim', 'cook','zuckerberg','ceo','tim cook ','steve ballmer','ballmer','elop',
      'schiller','fadell','phil spencer','spencer','mcspirit','sandberg','paul','allen','larry hryb','hryb']
      

types=['tablet','mobile', 'laptop', 'pc', 'computer', 'desktop','smartphone', 'smartwatch', 'search engine', 'software','hardware',
               'machine', 'browser','ebook', 'book',  'reader',  'console', 'headphone', 'earbud','bud','store','music',
              'gaming', 'operating','streaming','title','chatbot']



competitor=['samsung', 'galaxy',  'twitter','tiktok', 'switch','sony', 'asos', 'activision blizzard', 'activision','blizzard',
            'nintendo','snes', 'netflix','android','yahoo','nokia','huawei','motorola','htc','blackberry','oppo','oneplus','rim','symbian','bbc','morrison','spotify'] 


configue=['device','feature','battery','screen','sound','gb','g','k','mm','chip','processor','design','display','touch','ram',
          'inch','keyboard','camera','handset','speaker','button','touchscreen','storage', 'data']


celebrity=['dubost','neymar','amanda','beyonce','blur','richard','hammond','ranj','jeremy clarkson', 'jeremy','momoa',
           'jared','aniston','smith','kim','tony','tom','sophie','oasis','trio','sharon','betty','raoul','moat','lauren','andrew',
           'samuel gibbs','samuel','gibbs','van','gaal']

topics = celebrity + configue + competitor + types + ceos + products + firms + consumer_protection + technology

On stem les topics : 

In [148]:
string_of_topics = ' '.join(topics)
string_of_topics = stemmer.stem(string_of_topics)
stem_topics = string_of_topics.split(' ')
'''stem_topics = set(stem_topics) c'est gadget mais je veux faire un douvle isin avec 2 listes 
df['stem_topics'] = stem_topics * df.shape[0]'''

"stem_topics = set(stem_topics) c'est gadget mais je veux faire un douvle isin avec 2 listes \ndf['stem_topics'] = stem_topics * df.shape[0]"

In [146]:
# df = df.loc[len(df['agenda'] & df['stem_topics']) > 0]

In [88]:
# c'est trop nul mais je n'arrive pas à l'améliorer 

def techno_filter(list):
  for k in range(len(list)):
    if list[k] in stem_topics :
      return True
  return False

In [ ]:
df['agenda']

In [90]:
df['agenda'] = df['agenda'].apply(techno_filter)

In [91]:
S = df.loc[df['agenda']].shape[0]
S

7890

On voit que sur les 5000 premiers speeches, seuls concernent les topics définis. C'est peu, mais plutôt logique vu que ces speeches ont été faits en 1988 environ. 

On ne garde que ceux-là :

In [20]:
df = df.loc[df['agenda']]
df.drop(columns=['agenda'], inplace=True)

In [21]:
df['text'] = df['text'].apply(clean)

/Users/ismailakrout/opt/anaconda3/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [22]:
# On fait le groupby() par speaker et party par la suite pour ne pas dépasser les 1 000 000 carc 
df = df.groupby(by=['Speaker', 'party']).sum().reset_index()

On va repérer les 100 mots (puis 500 quand on aura les speeches en entier) qui sont le plus utilisés par chaque parti, et noter leur fréquence d'apparition. 

In [23]:
aux = df[['party', 'text']]

In [24]:
aux.groupby(by=['party']).sum()

,text
party,
Con,"[welsh, produc, welsh, lamb, beef, particular,..."
Lab,"[pleasur, hon, learn, kensington, sir, malcolm..."


In [25]:
def words_of_party(df, text):
  l = []
  for i in df.index: 
    if df['party'][i] == text :
      l = l + df['text'][i]
  return l

In [26]:
words_of_Con = words_of_party(df, 'Con')
words_of_Lab = words_of_party(df, 'Lab')
# words_of_LibDem = words_of_party(df, 'LibDem')
# words_of_SDLP = words_of_party(df, 'SDLP')

In [27]:
def words_frequencies(list):
  words = []
  freqs = []
  for word in list: 
    if word not in words : 
      words.append(word)
      freqs.append(1)
    else :
      k = words.index(word)
      freqs[k] += 1
  d = {'words': words, 'freqs': freqs}
  words_freqs =  pd.DataFrame(data=d)
  return words_freqs

In [28]:
words_frequencies(words_of_Con)

,words,freqs
0,welsh,2
1,produc,3
2,lamb,1
3,beef,1
4,particular,2
...,...,...
699,thwart,1
700,intellig,1
701,trust,1
702,expos,1


On a tous les mots utilisés par le parti Conservateur sur les sujets définis, ainsi que leurs fréquences d'apparition. On va retenir les fréquences avec lesquelles chaques mots sont utilisés, par chaque parti, afin de déterminer le chi2 de chaque mot. On se restreint pour l'instant aux partis Con et Lab, fortement majoritaires. 

In [29]:
df_freqs = pd.merge(words_frequencies(words_of_Con), words_frequencies(words_of_Lab), on="words", how='outer')

In [30]:
c = 0
for i in range(len(df_freqs['words'])):
  if df_freqs['freqs_x'][i] > 0 and df_freqs['freqs_y'][i] >0 :
    c += 1

In [31]:
c

201

Il y a donc c mots utilisés par les deux partis. 

In [32]:
df_freqs

,words,freqs_x,freqs_y
0,welsh,2.0,NaN
1,produc,3.0,NaN
2,lamb,1.0,NaN
3,beef,1.0,NaN
4,particular,2.0,1.0
...,...,...,...
1081,faith,NaN,1.0
1082,everyon,NaN,1.0
1083,welfar,NaN,1.0
1084,nurseri,NaN,2.0


In [33]:
# Correction du test du Qi 2 

df_freqs.loc[df_freqs['freqs_x'].notna(), 'fp_Con'] = df_freqs.loc[df_freqs['freqs_x'].notna(), 'freqs_x']
df_freqs['fp_Con'] = df_freqs['fp_Con'].fillna(0)

df_freqs.loc[df_freqs['freqs_y'].notna(), 'fp_Lab'] = df_freqs.loc[df_freqs['freqs_y'].notna(), 'freqs_y']
df_freqs['fp_Lab'] = df_freqs['fp_Lab'].fillna(0)

s_con = df_freqs['fp_Con'].sum()
df_freqs['not_fp_Con'] = s_con - df_freqs['fp_Con']

s_lab = df_freqs['fp_Lab'].sum()
df_freqs['not_fp_Lab'] = s_lab - df_freqs['fp_Lab']

df_freqs['chi_2'] = ((df_freqs['fp_Con'] * df_freqs['not_fp_Con'] - df_freqs['fp_Lab'] * df_freqs['not_fp_Lab'])**2) / ((df_freqs['fp_Con']+df_freqs['fp_Lab']) * (df_freqs['not_fp_Lab']+df_freqs['fp_Lab']) * (df_freqs['not_fp_Con']+df_freqs['fp_Con']) * (df_freqs['not_fp_Con']+df_freqs['not_fp_Lab']))

df_freqs.drop(columns=['fp_Con', 'fp_Lab', 'not_fp_Con', 'not_fp_Lab'], inplace=True)

In [34]:
df_freqs

,words,freqs_x,freqs_y,chi_2
0,welsh,2.0,NaN,0.001194
1,produc,3.0,NaN,0.001790
2,lamb,1.0,NaN,0.000598
3,beef,1.0,NaN,0.000598
4,particular,2.0,1.0,0.000346
...,...,...,...,...
1081,faith,NaN,1.0,0.000277
1082,everyon,NaN,1.0,0.000277
1083,welfar,NaN,1.0,0.000277
1084,nurseri,NaN,2.0,0.000554


On a donc ici chaque mot utilisé par les parti sur le sujet délimité, avec le chi2 correspondant. On va se limiter aux 500 premiers mots :

In [35]:
df_freqs = df_freqs.sort_values(by=['chi_2'], ascending=False)

In [36]:
df_freqs = df_freqs.head(500)

In [37]:
df_freqs

,words,freqs_x,freqs_y,chi_2
20,hon,38.0,13.0,0.009575
21,friend,23.0,4.0,0.008895
595,rail,9.0,NaN,0.005339
304,network,9.0,NaN,0.005339
24,cours,8.0,NaN,0.004750
...,...,...,...,...
195,ill,1.0,NaN,0.000598
99,reconcil,1.0,NaN,0.000598
103,prepar,1.0,NaN,0.000598
192,die,1.0,NaN,0.000598


In [38]:
# là on obtient la liste des 500 mots les plus importants 
lili = df_freqs['words'].unique()

# On forme le tableau 

In [39]:
lili

array(['hon', 'friend', 'rail', 'network', 'cours', 'consid', 'govern',
       'programm', 'busi', 'hub', 'job', 'econom', 'announc', 'economi',
       'decis', 'countri', 'veri', 'charter', 'achiev', 'disabl', 'deliv',
       'rais', 'quick', 'quit', 'increas', 'ladi', 'parti', 'possibl',
       'european', 'northern', 'import', 'befor', 'continu', 'introduc',
       'express', 'engag', 'absolut', 'nato', 'appreci', 'railway',
       'shipbuild', 'packag', 'rout', 'estim', 'industri', 'negoti',
       'addit', 'regul', 'benefit', 'creat', 'improv', 'propos', 'employ',
       'parliament', 'respons', 'labour', 'court', 'opportun', 'argument',
       'uk', 'valley', 'cp', 'upgrad', 'highspe', 'grow', 'hlos', 'fear',
       'membership', 'taxpay', 'judg', 'ship', 'energi', 'climat',
       'origin', 'execut', 'skill', 'produc', 'rule', 'unemploy',
       'competit', 'electrif', 'aid', 'bbc', 'proud', 'advoc', 'element',
       'passeng', 'transport', 'extrem', 'potenti', 'congratul', 'fi

In [40]:
'''
# La c'est bien codé
def count_liste(lili, mot):
    return lili.count(mot)
for word in lili:
    df[f'{word}'] = df['text'].apply(count_liste, args=(word,))
'''

"\n# La c'est bien codé\ndef count_liste(lili, mot):\n    return lili.count(mot)\nfor word in lili:\n    df[f'{word}'] = df['text'].apply(count_liste, args=(word,))\n"

In [41]:
# tres mal codé
for word in lili:
    def count_liste(lili):
        return lili.count(word)
    df[f'{word}'] = df['text'].apply(count_liste)


/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_30223/451958271.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{word}'] = df['text'].apply(count_liste)
/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_30223/451958271.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{word}'] = df['text'].apply(count_liste)
/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_30223/451958271.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` m

In [42]:
df['somme'] = df.sum(axis=0)

/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_30223/1347020148.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['somme'] = df.sum(axis=0)


In [43]:
a = np.array([[1,2,3],
              [4,5,6],
              [7,8,9]])

In [44]:
a.sum(axis=0)

array([12, 15, 18])

In [45]:
df['somme'] = df.sum(axis=1)
df.loc[df['somme'] == 0, 'somme'] = 1

/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_30223/642274951.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['somme'] = df.sum(axis=1)


In [46]:
df['somme']

0      38
1       1
2      16
3     114
4      12
5       9
6       2
7     198
8      44
9      10
10     26
11     38
12      8
13      9
14     13
15     12
16      5
17     24
18     29
19      7
20     14
21     18
22     40
23     14
24     26
25      4
26     16
27     44
28     19
29     26
30      1
31     30
32      9
33     32
34     84
35    279
36     19
37      9
38      2
39      7
Name: somme, dtype: int64

In [47]:
for word in lili:
    df[f'{word}'] = df[f'{word}'] / df['somme'] 

In [48]:
df

,Speaker,party,text,hon,friend,rail,network,cours,consid,govern,programm,busi,hub,job,econom,announc,economi,decis,countri,veri,charter,achiev,disabl,deliv,rais,quick,quit,increas,ladi,parti,possibl,european,northern,import,befor,continu,introduc,express,engag,absolut,nato,appreci,railway,shipbuild,packag,rout,estim,industri,negoti,addit,regul,benefit,creat,improv,propos,employ,parliament,respons,labour,court,opportun,argument,uk,valley,cp,upgrad,highspe,grow,hlos,fear,membership,taxpay,judg,ship,energi,climat,origin,execut,skill,produc,rule,unemploy,competit,electrif,aid,bbc,proud,advoc,element,passeng,transport,extrem,potenti,congratul,fiscal,establish,fee,specif,worri,licenc,recognis,motion,judici,amend,remind,nation,sector,awar,citi,assur,fulli,penkridg,happi,tran,convinc,pennin,littl,wider,success,greenhous,gase,smes,workplac,ptes,determin,grate,encourag,greater,effect,write,sovereignti,context,creation,breakthrough,english,baltic,implement,fleet,vital,fewer,input,parent,probabl,prais,fundament,scottish,agenda,welsh,coln,prioriti,census,complement,websit,inform,y,aim,taxfre,assess,birmingham,hs,russian,evalu,sustain,crowd,guardian,limit,enterpris,primaci,signific,guid,develop,unlik,enunci,privat,malaria,sourc,reduc,easier,tackl,thatth,overal,figur,explor,adopt,news,unit,kingdom,primarili,pii,royal,precis,strategi,reform,dither,answer,challeng,sinc,israel,compani,colleagu,broken,isra,secretari,constitu,issu,fund,prime,gaza,civil,chair,asid,explan,leav,deficit,sometim,tori,liberti,insur,refus,speak,realli,nuclear,blockad,privaci,document,weapon,promot,exampl,law,financ,obvious,renew,decid,shadow,capac,duti,refer,claus,union,england,polici,suggest,avail,gentleman,debat,futur,sever,scott,envisag,permit,damag,churchil,toe,els,favour,confus,vindic,matrix,abroad,crisi,regardless,certif,clackmannanshir,geotherm,bencher,protocol,expens,necessarili,recommend,technic,relat,flow,restat,boost,lincolnshir,kimberlyclark,closur,inadvert,keyn,sensibl,legitim,factori,lobbi,buckingham,immigr,bartonuponhumb,themselv,persuad,materi,secret,everyth,approv,perhap,attract,medic,clinician,distinct,client,auxiliari,minor,paper,analysi,centro,leg,leed,manchest,goahead,natur,infrastructur,scope,ineffici,thus,command,relev,becom,mcnulti,ray,compet,overstretch,elsewher,depend,runup,disappoint,afraid,preempt,midland,strengthen,spread,minut,expos,thwart,oblig,thrust,control,evid,commend,balanc,hexham,prioritis,woodhead,reopen,stocksbridg,peniston,appropri,huddersfield,stalybridg,station,slaithwait,marsden,greenfield,mossley,pick,individu,somewhat,actual,denbi,activ,scotland,traumat,spirit,steadfast,arrest,swift,darci,jason,nineyearold,injuri,louisa,eventu,stab,terribl,issueâ,unlimit,âwe,pop,leveson,danger,stick,timet,slip,offici,experi,dure,phase,thorough,itbecaus,projectal,valueformoney,percept,champion,pension,confid,expertis,necessari,shipyard,capabl,solid,procur,celebr,workforc,invest,navi,framework,feasibl,commiss,hmrc,broad,milton,respond,rapid,guidanc,nonsens,sectionâi,deepli,suffragett,shoulder,regist,equiti,beef,capit,pym,outâ,âone,corpor,hampden,motiona,buri,organis,nuttal,defin,count,claimant,morn,othersbut,virtu,section,pursu,perform,hunt,gdphas,compon,investmenta,profess,fiction,complet,implic,onward,microbusi,rid,oneout,onein,internet,tape,overregul,goldplat,backyard,assert,suprem,ad,statesâth,pool,progress,advanc,insert,impos,platform,caveat,condemn,requir,inspir,brilliant,hitchin,priori,headteach,edward,geraint,alleg,democrat,ultim,embark,accountabilityand,parliamentari,lewi,lock,carrolltyp,âthe,phraseolog,recal,extremist,length,exclus,lamb,net,bed,malari,inconsist,background,nutshel,ill,reconcil,prepar,die,âfiscal,somme
0,Alan Johnson,Lab,"[pleasur, hon, learn, kensington, sir, malcolm...",0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [49]:
df.loc[df['party'] == 'Con', 'party_bool'] = 1
df.loc[df['party'] == 'Lab', 'party_bool'] = 0

/var/folders/q8/b8575nd94453wggnc43r1xbc0000gn/T/ipykernel_30223/3670770691.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[df['party'] == 'Con', 'party_bool'] = 1


# Regressions linéaires

In [50]:
import statsmodels.api as sm

dico={}
dico['Speaker'] = ['intercept', 'slope_coefficient']

for word in lili:
    x = df['party_bool']
    y = df[f'{word}']

    # adding the constant term
    x = sm.add_constant(x)

    result = sm.OLS(y, x).fit()

    dico[f'{word}'] = [result.params[0], result.params[1]]


In [51]:
aux = pd.DataFrame(dico)
df = pd.concat([df, aux]).reset_index(drop=True)

In [52]:
df.drop(columns=['somme'], inplace=True)
df = df[['Speaker', 'party', 'text']+['party_bool'] + list(df.columns)[3:-1]]

In [53]:
df

,Speaker,party,text,party_bool,hon,friend,rail,network,cours,consid,govern,programm,busi,hub,job,econom,announc,economi,decis,countri,veri,charter,achiev,disabl,deliv,rais,quick,quit,increas,ladi,parti,possibl,european,northern,import,befor,continu,introduc,express,engag,absolut,nato,appreci,railway,shipbuild,packag,rout,estim,industri,negoti,addit,regul,benefit,creat,improv,propos,employ,parliament,respons,labour,court,opportun,argument,uk,valley,cp,upgrad,highspe,grow,hlos,fear,membership,taxpay,judg,ship,energi,climat,origin,execut,skill,produc,rule,unemploy,competit,electrif,aid,bbc,proud,advoc,element,passeng,transport,extrem,potenti,congratul,fiscal,establish,fee,specif,worri,licenc,recognis,motion,judici,amend,remind,nation,sector,awar,citi,assur,fulli,penkridg,happi,tran,convinc,pennin,littl,wider,success,greenhous,gase,smes,workplac,ptes,determin,grate,encourag,greater,effect,write,sovereignti,context,creation,breakthrough,english,baltic,implement,fleet,vital,fewer,input,parent,probabl,prais,fundament,scottish,agenda,welsh,coln,prioriti,census,complement,websit,inform,y,aim,taxfre,assess,birmingham,hs,russian,evalu,sustain,crowd,guardian,limit,enterpris,primaci,signific,guid,develop,unlik,enunci,privat,malaria,sourc,reduc,easier,tackl,thatth,overal,figur,explor,adopt,news,unit,kingdom,primarili,pii,royal,precis,strategi,reform,dither,answer,challeng,sinc,israel,compani,colleagu,broken,isra,secretari,constitu,issu,fund,prime,gaza,civil,chair,asid,explan,leav,deficit,sometim,tori,liberti,insur,refus,speak,realli,nuclear,blockad,privaci,document,weapon,promot,exampl,law,financ,obvious,renew,decid,shadow,capac,duti,refer,claus,union,england,polici,suggest,avail,gentleman,debat,futur,sever,scott,envisag,permit,damag,churchil,toe,els,favour,confus,vindic,matrix,abroad,crisi,regardless,certif,clackmannanshir,geotherm,bencher,protocol,expens,necessarili,recommend,technic,relat,flow,restat,boost,lincolnshir,kimberlyclark,closur,inadvert,keyn,sensibl,legitim,factori,lobbi,buckingham,immigr,bartonuponhumb,themselv,persuad,materi,secret,everyth,approv,perhap,attract,medic,clinician,distinct,client,auxiliari,minor,paper,analysi,centro,leg,leed,manchest,goahead,natur,infrastructur,scope,ineffici,thus,command,relev,becom,mcnulti,ray,compet,overstretch,elsewher,depend,runup,disappoint,afraid,preempt,midland,strengthen,spread,minut,expos,thwart,oblig,thrust,control,evid,commend,balanc,hexham,prioritis,woodhead,reopen,stocksbridg,peniston,appropri,huddersfield,stalybridg,station,slaithwait,marsden,greenfield,mossley,pick,individu,somewhat,actual,denbi,activ,scotland,traumat,spirit,steadfast,arrest,swift,darci,jason,nineyearold,injuri,louisa,eventu,stab,terribl,issueâ,unlimit,âwe,pop,leveson,danger,stick,timet,slip,offici,experi,dure,phase,thorough,itbecaus,projectal,valueformoney,percept,champion,pension,confid,expertis,necessari,shipyard,capabl,solid,procur,celebr,workforc,invest,navi,framework,feasibl,commiss,hmrc,broad,milton,respond,rapid,guidanc,nonsens,sectionâi,deepli,suffragett,shoulder,regist,equiti,beef,capit,pym,outâ,âone,corpor,hampden,motiona,buri,organis,nuttal,defin,count,claimant,morn,othersbut,virtu,section,pursu,perform,hunt,gdphas,compon,investmenta,profess,fiction,complet,implic,onward,microbusi,rid,oneout,onein,internet,tape,overregul,goldplat,backyard,assert,suprem,ad,statesâth,pool,progress,advanc,insert,impos,platform,caveat,condemn,requir,inspir,brilliant,hitchin,priori,headteach,edward,geraint,alleg,democrat,ultim,embark,accountabilityand,parliamentari,lewi,lock,carrolltyp,âthe,phraseolog,recal,extremist,length,exclus,lamb,net,bed,malari,inconsist,background,nutshel,ill,reconcil,prepar,die,âfiscal
0,Alan Johnson,Lab,"[pleasur, hon, learn, kensington, sir, malcolm...",0.0,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [54]:
df.to_csv('01. output/df_500_with_slope_coef.csv')